In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error

## Neocis data##

In [40]:
df = pd.read_csv('https://raw.githubusercontent.com/bevi-rosso/Neocis/master/Neocis.csv', sep=';')
X = np.array(df.index)
x = df.drop(['Mooney','VEI'],axis=1)
y = df.Mooney
print ('Dati in input = ', len(df))
df.head()

Dati in input =  38


,Tingresso,Solvente,Nd/PB,Al/Nd,S/Bde,Mooney,VEI
0,29.4663,30003.6,1.96452,5.88494,6.00264,41.2,0.91
1,29.3233,30008.8,1.96855,5.87461,6.00106,40.8,0.91
2,29.2119,29999.2,1.98207,5.82938,5.99960,38.8,0.92
3,29.2523,30012.4,1.98238,5.79447,6.00642,38.6,0.88
4,32.1935,30009.8,1.98835,5.79098,6.00157,37.3,0.88


## PCA ##

In [3]:
pca = PCA()
xs = scale(df)
x_red = pca.fit_transform(xs)
V = pca.components_
nPC = np.arange(len(V)) + 1
dfV = pd.DataFrame(V, columns=[df.columns], index=[nPC])
EV = pd.DataFrame(np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100),\
                  columns =['Explained Variance'], index=[nPC])

In [4]:
print ('PCs = ', len(V))
dfV

PCs =  7


,Tingresso,Solvente,Nd/PB,Al/Nd,S/Bde,Mooney,VEI
1,-0.358783,-0.418246,0.329424,-0.414178,-0.363168,0.350172,0.402206
2,0.449975,-0.174692,-0.563403,-0.084179,-0.372736,0.516521,-0.191753
3,0.408843,0.140916,0.443297,-0.407815,0.503850,0.408207,-0.172243
4,-0.138822,-0.080493,-0.410253,0.182902,0.587020,0.312900,0.574447
5,0.668456,-0.284311,0.290850,0.317304,-0.121018,-0.221395,0.472649
6,0.010482,0.799333,0.124231,0.161756,-0.340413,0.301865,0.335195
7,-0.188228,-0.220116,0.330470,0.703390,0.013905,0.453771,-0.325653


In [5]:
EV

,Explained Variance
1,73.34
2,87.23
3,92.85
4,96.04
5,98.30
6,99.38
7,100.01


In [83]:
col = [x for x in range(len(X))]
fig=go.Figure()
fig.add_trace(go.Scatter(x=x_red[:,0], y=x_red[:,1], mode='markers', marker=dict(symbol=[200], \
             color=col, line_width=2, size=10)))
fig.update_layout(title='PCA - Score plot', xaxis_title='PC1',yaxis_title='PC2', xaxis_zeroline=True, yaxis_zeroline=True)

In [32]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=X, y=x_red[:,0], mode='lines', name='PC1', marker_color = 'blue'))
fig.add_trace(go.Scatter(x=X, y=x_red[:,1], mode='lines', name='PC2', marker_color = 'red'))
fig.update_layout(title='PCA', xaxis_title='n° samples',yaxis_title='PC1, PC2', xaxis_zeroline=True)

In [41]:
arr = np.array(dfV)
label = np.array(df.columns)
plot = go.Scatter(x=arr[0,:], y=x_red[1,:], mode='markers', marker=dict(symbol=[201],color='blue', size=10), \
              text = label)
fig=go.Figure()
fig.add_trace(plot)
fig.update_layout(title='PCA - Loadings plot', xaxis_title='PC1',yaxis_title='PC2', xaxis_zeroline=True, yaxis_zeroline=True)